In [260]:
%matplotlib ipympl
from impedance.models.circuits import CustomCircuit
from impedance.models.circuits.elements import element
from impedance.visualization import plot_nyquist
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

In [261]:
@element(num_params=4, units=["Ohm", "Ohm", "F", ""],overwrite=True)
def TLMtwoNew(p, f):
   
    omega = 2 * np.pi * np.array(f)
    R1, R2, Q, delta1 = p[0], p[1], p[2], p[3]
    delta2=1-delta1
    k12=R2/R1
    CTotal1=Q
    CTotal2=Q
    wc1=1/(R1*CTotal1)
    wc2=1/(R2*CTotal2)
    S1=np.sqrt(omega*1j/wc1)
    S2=np.sqrt(omega*1j/wc2)
    C1=1/np.tanh(delta1*S1)
    C2=1/np.tanh(delta2*S2)

    Z=2*(R1)*(C1*C2*S1*k12+S2)/(S1*(C2*S1*k12+C1*S2))

    return Z

In [262]:
def plotter_and_saver(fitted_pm,pm_conf,input_pm,freq,save_folder_name):
  
    R1_fit=fitted_pm[0]
    R2_fit=fitted_pm[1]
    Q_fit=fitted_pm[2]
    delta_fit=fitted_pm[3]

    R1_in=input_pm[0]
    R2_in=input_pm[1]
    Q_in=input_pm[2]
    delta_in=input_pm[3]

    circuit_in=CustomCircuit(initial_guess=[R1_in,R2_in,Q_in,delta_in],circuit='TLMtwoNew_0')
    z_in=circuit_in.predict(freq)
    circuit_fit=CustomCircuit(initial_guess=[R1_fit,R2_fit,Q_fit,delta_fit],circuit='TLMtwoNew_0')
    z_fit=circuit_fit.predict(freq)

    linewidth=2
    ticksize=6
    color_actual="blue"
    color_fitted="green"

    fig = plt.figure(f"Results at δ={delta_in}")
    
    ax1= fig.add_subplot(2,2,1)

    ax1.plot([0,delta_in,delta_in,1],[R1_in,R1_in,R2_in,R2_in],linewidth=linewidth,color=color_actual,label="Actual")
    ax1.plot([0,delta_fit,delta_fit,1],[R1_fit,R1_fit,R2_fit,R2_fit],linewidth=linewidth,color=color_fitted,label="Fitted")
    ax1.tick_params(labelsize=ticksize)
    ax1.axvline(x=delta_in,ls="--",linewidth=0.6,color=color_actual)
    ax1.axvline(x=delta_fit,ls="--",linewidth=0.6,color=color_fitted)

    ax1.set_ylabel("R1 / R2 (Ω)")
    ax1.set_xlabel("δ")
    ax1.set_xmargin(0)
    ax1.legend()

    ax2=fig.add_subplot(2,2,2)
    ax2.set_xlim(0,1)
    ax2.set_ylim(0,1)
    plot_nyquist(z_in/1000,ax=ax2,units="kΩ",labelsize=10,fmt="",color=color_actual,linewidth=linewidth,ticksize=ticksize)
    plot_nyquist(z_fit/1000,ax=ax2,units="kΩ",labelsize=10,fmt="",color=color_fitted,linewidth=linewidth,ticksize=ticksize)

    

    fitted_pm=np.around(fitted_pm,decimals=6,out=None)
    pm_conf=np.around(pm_conf,decimals=10,out=None)

    data =np.array([input_pm,
            fitted_pm,pm_conf])
    #cell_colors=[["blue","blue","blue","blue"],["green","green","green","green"]]
    

    # plt.figure(frameon=False)
    ax3=fig.add_subplot(4,1,4)
    ax3.set_axis_off()
    pm_table=ax3.table(data,rowLabels=["Actual","Fitted","S.D. error"],colLabels=["R1","R2","Q","δ"],loc="center")
    pm_table.auto_set_font_size(False)
    pm_table.set_fontsize(10)

    plt.savefig(save_folder_name+f"//δ={delta_in}.png",format="png")
    plt.tight_layout(w_pad=2,h_pad=7)
    #plt.show()
    plt.close()

In [ ]:
freq = np.logspace(3,-3,num=100)

R1_R2_in_list=((1000,100),(502,500))
delta_to_be_tested=[0.01,0.1,0.2,0.5,0.8,0.9,0.99]
#delta_to_be_tested=[0.2]
Q_in=6.61e-4
add_noise=True

### intially guessed values of parameters for the model ### 
R1_gs=300
R2_gs=300
delta_gs=0.5
Q_gs=Q_in

### creating a circuit object for the initial guess model (for which parameters are just above defined) ###
circuit_guess=CustomCircuit(initial_guess=[R1_gs,R2_gs,Q_gs,delta_gs],circuit='TLMtwoNew_0')

for R1_R2 in R1_R2_in_list:

    R1_in=R1_R2[0]
    R2_in=R1_R2[1]
    
    name_of_folder_to_save_data=f"R1={R1_in} R2={R2_in} Q={Q_in} and diff delta"

    if os.path.exists(name_of_folder_to_save_data):
        shutil.rmtree(name_of_folder_to_save_data)
    os.mkdir(f"{name_of_folder_to_save_data}")

    for delta_in in delta_to_be_tested:
        circuit_in=CustomCircuit(initial_guess=[R1_in,R2_in,Q_in,delta_in],circuit='TLMtwoNew_0')
        pm_input=[R1_in,R2_in,Q_in,delta_in]
        z_from_circuit_in = circuit_in.predict(freq)
        print(f"Before {z_from_circuit_in}")
        if add_noise:
            for i in range(len(z_from_circuit_in)):
                z_real_new=np.random.normal(np.real(z_from_circuit_in[i]),np.abs(0.05*np.real(z_from_circuit_in[i])))
                z_imag_new=np.random.normal(np.imag(z_from_circuit_in[i]),np.abs(0.05*np.imag(z_from_circuit_in[i])))
                z_from_circuit_in[i]=z_real_new+1j*z_imag_new
            
        print(f"After {z_from_circuit_in}")
        circuit_fitted=circuit_guess.fit(freq,z_from_circuit_in,bounds=(0,[np.inf,np.inf,np.inf,1]))
        pm_fitted=circuit_fitted.parameters_
        pm_conf=circuit_fitted.conf_
        plotter_and_saver(pm_fitted,pm_conf,pm_input,freq,name_of_folder_to_save_data)

In [264]:
#code for weighing

#freq=f_pred
#data=tlmuni_pred
#realsqr=np.real(data)**2
#imagsqr=np.imag(data)**2
#sgm_prop=np.concatenate((np.real(data),np.imag(data)))
#sgm_mod=np.concatenate((np.sqrt(realsqr+imagsqr),np.sqrt(realsqr+imagsqr)))
#sgm_unit=np.concatenate((np.ones(len(freq)),np.ones(len(freq))))